# ENSO impact on river discharge using GP Regression

This jupyter notebook was made in the course of a Bachelor's thesis in the degree of mediainformatics at the University of Tuebingen. 
This project evolved under the supervision of Bedartha Goswami and his research group "Machine Learning in Climate Science" at the Cluster Of Excellence "Machine Learning".

The scope of this project is to investigate ENSO impact on river discharge in South America using a data-driven approach. 
The data used for this project is provided by Do et. al via the 2018 released "Global Streamflow and Metadata and Indices Archive" (GSIM). 

With the GSIM dataset and Gaussian Process Regression we inferred missing values for stations to extend the temporal and spatial coverage of our analysis

## Import of the GSIM dataset

In [1]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
import GPy
import csv
import pickle

from datetime import datetime
from sklearn.preprocessing import normalize
from sklearn.metrics import mean_squared_error
from scipy import stats
from scipy.stats import norm, percentileofscore
from statistics import mean 

In [2]:
'''
Loads the meta, monhtly, yearly and seasonal data from the GSIM dataset for a given gsim number
To each dataframe the gsim number is added for further identification.
Colum names are cleaned and a list of all dataframes is finally returned
'''
def loadTimeseries (gsimNo): 
    meta = gsim_metadata.loc[gsim_metadata['gsim.no'] == gsimNo]
    monthly = pd.read_csv("data/GSIM_indices/TIMESERIES/monthly/"+gsimNo+".mon", sep=',' '\\t', skiprows=lambda x : x < 21, engine='python',)
    yearly = pd.read_csv("data/GSIM_indices/TIMESERIES/yearly/"+gsimNo+".year", sep=',' '\\t', skiprows=lambda x : x < 21, engine='python',)
    seasonal = pd.read_csv("data/GSIM_indices/TIMESERIES/seasonal/"+gsimNo+".seas", sep=',' '\\t', skiprows=lambda x : x < 21, engine='python',)
    monthly['gsim.no'] = gsimNo
    yearly['gsim.no'] = gsimNo
    seasonal['gsim.no'] = gsimNo
    monthly.columns, yearly.columns, seasonal.columns = monthly.columns.str.replace('"', ''), yearly.columns.str.replace('"', ''), seasonal.columns.str.replace('"', '')
    return [meta, monthly, yearly, seasonal]

In [3]:
'''
Transforms the last date of measurment for a season priveded in the gsim dataset as the feature date
As the seasonal gsim dataset stores the last date of measurement the last day of measurement of season DJF is in February, MAM in May and so on 
'''
def determimeSeason(date):
    endMonth = pd.to_datetime(date).month
    switcher = {
        2: "DJF",
        5: "MAM",
        8: "JJA",
        11: "SON"
    }
    return switcher.get(endMonth, "undefined")

In [4]:
# Loading the GSIM_metadata_Archive
gsim_metadata = pd.read_csv("data/GSIM_metadata/GSIM_catalog/GSIM_metadata.csv", sep=",")


# Selecting all the dataframes for South America by 2 Letter Country Code
southAmerica2LetterCC = ['AR', 'BO', 'BR', 'CL', 'CO', 'EC', 'FK', 'GF', 'GY', 'PY', 'PE', 'SR', 'UY', 'VE']

stations_of_interest = pd.Series()
for country in southAmerica2LetterCC: 
    stations_of_interest = stations_of_interest.append(gsim_metadata.loc[gsim_metadata['country'] == country]['gsim.no'], ignore_index=True)
        
    
# Initialisation of the empty lists where we store the dataframes: datatype: list of dataframes
df_yearly = []
df_monthly = []
df_seasonal = []
df_meta = []

# For each station in South America we load the meta, yearly, seasonal and monthly dataset
for station in stations_of_interest:
    
    series = loadTimeseries(station)
    df_meta.append(series[0])
    df_monthly.append(series[1])
    df_yearly.append(series[2])
    df_seasonal.append(series[3])

    
        
# For each dataframe in our monthly dataset we add a column that stores each the year and the month for a given measurement
for df in df_monthly: 
    df['month'] = pd.to_datetime(df['date']).dt.month
    df['year'] = pd.to_datetime(df['date']).dt.year
    df['date'] = pd.to_datetime(df['date']).dt.date

    
# For each dataframe in our yearly dataset we add a column that stores the year for a given measurement
for df in df_yearly: 
    df['year'] = pd.to_datetime(df['date']).dt.year
    df['date'] = pd.to_datetime(df['date']).dt.date


# For each dataframe in our seasonal dataset we add a column that stores the year and the season of the measurment. 
# Note that for the season DJF, the year is year of the months January and February is stored. 
for df in df_seasonal: 
    df['date'] = pd.to_datetime(df['date']).dt.date
    df['year'] = pd.to_datetime(df['date']).dt.year
    df['month'] = pd.to_datetime(df['date']).dt.month
    df['season'] = [determimeSeason(date) for date in df['date']]
    
    
# Merge the list of metadataframes into one single dataframe, as for each station only one row of metadat exists we can merge them into one single dataframe
for index,meta in enumerate(df_meta):
    if index == 0:
        ds_meta = meta
    else: 
        ds_meta = ds_meta.append(meta, ignore_index = True)

## Initialisation of functions and variables

In [5]:
# Initialisation of variables

START = datetime.strptime('1960-01-01', "%Y-%m-%d").date()
END =  datetime.strptime('2016-05-31', "%Y-%m-%d").date()
NUMBER_OF_DIMENSIONS = 10
TRAIN_TEST_THRESHOLD = 0.70
SAMPLESIZE = 1000
MIN_MONTHS_OF_OVERLAP = 60
MIN_YEARS_OF_OVERLAP = 5

In [6]:
''' 
Build the basis for the prediciton. 
Filters all the timeseries that initially cover the timespan defined between start and end.
The prediction_base is a list that contains all the timeseries that cover the timespan fully and will therefore be used for the correlation analysis with the timeseries we want to infer data for
@param list of dataframes
@returns a filtered list of dataframes that initially cover the whole timespan
'''

def buildPredictionBase (dataset):
    prediction_base = []
    for df in dataset:
        df_start = df['date'].iloc[0]
        df_end = df['date'].iloc[-1]
        if ((df_start <= START) and (df_end >= END)):                                                     # check if the first entry of this df is before or equal to the start and the end is after or equal to our end of our timeperiode 
            df_filtered = df.loc[(df['date'] >= START) & (df['date'] <= END)].reset_index(drop=True)      # Then select the timespan we defined above and store this df in our list 
            prediction_base.append(df_filtered)
        else:
            continue
 
    return prediction_base

In [7]:
'''
Check if a dataframe shows at least the minimum amount of months with available data during our timespan of interest
@param dataframe
@returns boolean
'''
def checkForMinimumTemporalOverlap (df):
    filterDf =  df.loc[(df['date'] >= START) & (df['date'] <= END)].reset_index(drop=True)
    arr = filterDf['MEAN'].to_numpy()
    arrNoNans = arr.copy()[~np.isnan(arr)]
    return arrNoNans.size >= MIN_MONTHS_OF_OVERLAP

In [8]:
'''
Gets all the timeseries that have at least fife years if temporal overlap with our previously defined timespan, but do not cover the full timespan. 
These are the timeseries we want to interpolate missing values for. 
'''
def getPredictableTimeseries (dataset, gsimNumbersPredictionBase):
    to_process = []
    for df in dataset:
        df_start = df['date'].iloc[0]
        df_end = df['date'].iloc[-1]
        df_gsimno = df['gsim.no'].values.any()
        
        if checkForMinimumTemporalOverlap(df):
            if (df_gsimno not in gsimNumbersPredictionBase):         # don't include the Timeseries that are already in our base
                df_filtered = df.loc[(df['date'] >= START) & (df['date'] <= END)]
                firstValidIndex = df_filtered['MEAN'].first_valid_index()
                lastValidIndex  = df_filtered['MEAN'].last_valid_index()
                df_cleaned = df_filtered.loc[firstValidIndex:lastValidIndex].reset_index(drop=True)
                to_process.append(df_cleaned)
            else: 
                continue
      
    return to_process

In [9]:
"""
Normalizes the data to a Gaussian distribution using quantiles. (function provided by Bedartha Goswami)
"""
def normalizeToGaussian(arr, mode="mean"):
    n = len(arr)
    perc = percentileofscore
    arr_ = arr.copy()[~np.isnan(arr)]
    out = np.zeros(n)
    for i in range(n):
        if not np.isnan(arr[i]):
            out[i] = norm.ppf(perc(arr_, arr[i], mode) / 100.)
        else:
            out[i] = np.nan
    return out


In [10]:
'''
In order to perform a spearmans correlation we need to truncate our prediction_base to the same timespan of our dataframe we want to process. 
This truncation happens here.
'''
def truncateBase (dataframe, base):
    start = dataframe['date'].iloc[0]
    end   = dataframe['date'].iloc[-1]
    filtered_base = [df.loc[(df['date'] >= start) & (df['date'] <= end)]  for df in base]
    
    return filtered_base

In [11]:
'''
Replaces missing values in the dataframe with the mean of the month for all years from which we have data
@param dataframe
@returns numpy array
'''
def averageNaNs (df):
    mean_df = df.groupby(['month']).mean()
    averaged_mean = df['MEAN'].copy().fillna(df['month'].map({1: mean_df['MEAN'][1] , 2: mean_df['MEAN'][2], 3:mean_df['MEAN'][3],
                                                    4: mean_df['MEAN'][4] , 5: mean_df['MEAN'][5], 6:mean_df['MEAN'][6], 
                                                    7: mean_df['MEAN'][7] , 8: mean_df['MEAN'][8], 9:mean_df['MEAN'][9],
                                                    10: mean_df['MEAN'][10] , 11: mean_df['MEAN'][11], 12:mean_df['MEAN'][12]}))
    
    averaged_numpy = averaged_mean.to_numpy()
    return averaged_numpy

In [12]:
'''
Computation of Spearmas Rank Correlation
@param  target_df: a pandas.dataframe (target) for wich we want to calculate the correlation matrix 
        dataset: a list of pandas dataframes that build our base, as we check for the correlation between dataframe and each df in the truncated dataset
@returns spearmans_matrix: a numpy.ndarray that contains the correlation value as first entry in each row and the corresponding p-value as a second element in each row. 
                            the order of stations in the matrix is consistent to the order in the dataset:
'''
def spearmansCorrelation (target_df, truncated_dataset): 
    spearmans_matrix = np.zeros((len(truncated_dataset), 2))
    for index, df in enumerate(truncated_dataset): 
        if (target_df.shape[0]!= df.shape[0]) or (df['MEAN'].isnull().all()):
            spearmans_matrix[index] = 0,100 
            continue
        else: 
            correlation, pval = stats.spearmanr(target_df['MEAN'], df['MEAN'], nan_policy='omit')
            spearmans_matrix[index] = correlation,pval    
            
    return spearmans_matrix

In [13]:
'''
Helper function to obtain the indices that show highest correlation 
@param ndarray
@returns numpy array with lenght NUMBER_OF_DIMENSIONS, containing the indices of the most correlated stations
'''
def getIndices (spearmans_matrix): 
    corr = spearmans_matrix[:,0].copy()
    indices = corr.argsort()[-NUMBER_OF_DIMENSIONS:][::-1]
    if indices.size < NUMBER_OF_DIMENSIONS:
        print("Less than {} timeseries while trying to get the indices".format(NUMBER_OF_DIMENSIONS))
    
    return indices

In [14]:
'''
Based on a dataframe for which we want to fill missing values we select the most correlated timeseries of our prediction base
Then we transform and truncate the time series so that we get input output mappings for training, and sets of input timeseries
for the time before and after the active timespan of our dataframe for which we want to infer missing data.
@param pandas.dataframe (target timeseries), list of pandas.dataframes (predictionBase), numpy.array (indices)
@returns multiple numpy arrays with training and test data, and the dates for the timesegments for training and prediction
'''
def getDataForTrainingAndPredicting (dataframe, prediction_base, indices): 
    
    X_train  = []
    X_test   = []
    X_before = []
    X_after  = []
    
    dim = indices.size
    start = dataframe['date'].iloc[0]
    end = dataframe['date'].iloc[-1]
    
    mostCorrelatedTimeseries = [prediction_base[index] for index in indices]
    
    splitind_df = int(dataframe.shape[0] * TRAIN_TEST_THRESHOLD)
   
    startind = max(0, mostCorrelatedTimeseries[0].loc[mostCorrelatedTimeseries[0]['date'] == start].index.values[0])
    splitind = startind + splitind_df 
    endind = min(mostCorrelatedTimeseries[0].shape[0] -1, mostCorrelatedTimeseries[0].loc[mostCorrelatedTimeseries[0]['date'] == end].index.values[0])
    
    
    datesBefore = mostCorrelatedTimeseries[0]['date'][:startind]
    datesTrain = mostCorrelatedTimeseries[0]['date'][startind :splitind]
    datesTest = mostCorrelatedTimeseries[0]['date'][splitind : endind]
    datesAfter = mostCorrelatedTimeseries[0]['date'][endind :]

    for ts in mostCorrelatedTimeseries: 
        tsNormalized = normalizeToGaussian(ts['MEAN'].copy().to_numpy())
        x_train = tsNormalized[startind: splitind].copy()
        
        x = normalizeToGaussian(averageNaNs(ts)).reshape(-1,1)
        x_before = x[:startind].copy()
        x_after = x[endind:].copy()
        x_test = x[splitind: endind].copy()
        
        X_train.append(x_train)
        X_test.append(x_test)
        X_before.append(x_before)
        X_after.append(x_after)

    X_test = np.column_stack((X_test))
    X_train = np.column_stack((X_train))
    X_before = np.column_stack((X_before))
    X_after = np.column_stack((X_after))
    
    Y_test = normalizeToGaussian(averageNaNs(dataframe))[splitind_df:].copy().reshape(-1,1)    
    Y_train  = normalizeToGaussian(dataframe['MEAN'].copy().to_numpy())[:splitind_df].reshape(-1,1) 

    train = np.concatenate((X_train,Y_train), axis=1)
    train_clean = train.copy()[~np.isnan(train).any(axis=1)]
    Y_train = train_clean[:,-1].reshape(-1,1)
    X_train = train_clean[:,:dim]

    return  X_before, X_after, X_train, X_test, Y_train, Y_test, datesBefore, datesTrain, datesTest, datesAfter

In [15]:
'''
Train a Gaussian Process model based on the training data
@param two numpy arrays of same dimensionality
@returns numpy array with lenght NUMBER_OF_DIMENSIONS, containing the indices of the most correlated stations
'''
def trainGP (X_train, Y_train):
    assert X_train.shape[0] == Y_train.shape[0], "Shape of training data is inconsistent"
    
    kernel = GPy.kern.Matern32(input_dim = NUMBER_OF_DIMENSIONS )
    model = GPy.models.GPRegression(X_train, Y_train, kernel)
    model.optimize(messages=True)
    model.optimize_restarts(num_restarts=10)
    
    return model

## Training GPs and inferring data

In [16]:
# Compute the basis for predicting or inferring data and its corresponding gsimNumbers
# Also those stations and timeseries that show missing values are filtered and saved in predictableTimeseries
# All of them are pickeld and saved on the hard drive such that we can use the same data in other notebooks 

predictionBase = buildPredictionBase(df_monthly)
gsimNumbersPredictionBase = [df['gsim.no'].values.any() for df in predictionBase]
predictableTimeseries = getPredictableTimeseries(df_monthly, gsimNumbersPredictionBase)


with open ('data/predictionBase.pkl', 'wb') as outp:
    pickle.dump(predictionBase, outp, pickle.HIGHEST_PROTOCOL )
    
with open ('data/predictableTimeseries.pkl', 'wb') as outp:
    pickle.dump(predictableTimeseries, outp, pickle.HIGHEST_PROTOCOL )
    
with open ('data/gsimNumbersPredictionBase.pkl', 'wb') as outp:
    pickle.dump(gsimNumbersPredictionBase, outp, pickle.HIGHEST_PROTOCOL )
    
   

In [ ]:
# WARNING this cell takes a couple of hours to run

# This code segment is performing the interpolation of missing data by using Gaussian Process Regression
# We iterate over each dataframe we want to infer data for
# the gsim number is stored seperately for indentification later on. 
# Our predictionBase is transformed(truncated) and then Spearmanns correlation is performed between the predcition base and the current dataframe
# Based on them we split those timeseries into training, test and precition data (before & after)
# Then we train the GP Regression model by feeding training and test data
# Later we predict the missing value with the training data from before and after the shared timespan. 
# Every part is then stored inseperate csv file. 

predictionError = []
predictionSuccess = []
df_error = np.array([99.99])
dfMissing = pd.DataFrame(data=[df_error], columns=['ERROR'])

for index, df in enumerate(predictableTimeseries): 
    print('We are at frame: {}'.format(index))
    gsimNo = df['gsim.no'].values.any()
    
    #caculate the most correlated timeseries with the current one
    truncatedBase = truncateBase (df, predictionBase)
    try: 
        spearmansMatrix = spearmansCorrelation(df, truncatedBase)
        indices = getIndices(spearmansMatrix)
        X_before, X_after, X_train, X_test, Y_train, Y_test, datesBefore, datesTrain, datesTest, datesAfter = getDataForTrainingAndPredicting(df, predictionBase, indices)
    except: 
        predictionError.append(gsimNo)
        print("Could not compute the SpearmansCorrelation at df {}".format(gsimNo))
    
    if (X_train.shape[0] != 0):
        try:
            model = trainGP(X_train, Y_train)
            meanTest, varTest = model.predict(X_test)
            meanBefore, varBefore = model.predict(X_before)
            meanAfter, varAfter = model.predict(X_after)
        except: 
            predictionError.append(gsimNo)
            print("Could not predict data for df: {}".format(gsimNo))
            
        try:                         # Transfer the prediction before available to a dataset and save it
            dfBefore = pd.DataFrame(data=meanBefore.T, columns=datesBefore)
            dfBefore_var = pd.DataFrame(data=varBefore.T, columns=datesBefore)
            dfBefore['gsim.no'] = gsimNo
            dfBefore_var['gsim.no'] = gsimNo
            dfBefore.to_csv('data/predictions/before/prediction{}_before.csv'.format(index))
            dfBefore_var.to_csv('data/predictions/before_var/prediction{0}_before.csv'.format(index))

        except: 
            predictionError.append(gsimNo)
            #print('An error occured at meanBefore with shape {1} in frame: {0}'.format(index, meanBefore.shape))
            print('Shape of datesBefore: {}'.format(datesBefore.shape))
                
        try:                         # Transfer the test prediction to a dataset and save it
            dfTest = pd.DataFrame(data=meanTest.T, columns=datesTest)
            dfTest_var = pd.DataFrame(data=varTest.T, columns=datesTest)
            dfTest['gsim.no'] = gsimNo
            dfTest_var['gsim.no'] = gsimNo
            dfTest.to_csv('data/predictions/test/prediction{}_test.csv'.format(index))
            dfTest_var.to_csv('data/predictions/test_var/prediction{}_test.csv'.format(index))
        except: 
            predictionError.append(gsimNo)
            #print('An error occured at meanTest with shape {1} in frame: {0}'.format(index, meanTest.shape))

        try:                         # Transfer the prediction after available data to a dataset and save it
            dfAfter = pd.DataFrame(data=meanAfter.T, columns=datesAfter)
            dfAfter_var = pd.DataFrame(data=varAfter.T, columns=datesAfter)
            dfAfter['gsim.no'] = gsimNo
            dfAfter_var['gsim.no'] = gsimNo
            dfAfter.to_csv('data/predictions/after/prediction{}_after.csv'.format(index))
            dfAfter_var.to_csv('data/predictions/after_var/prediction{}_after.csv'.format(index))
        except:
            predictionError.append(gsimNo)
            #print('An error occured at meanAfter with shape {1} in frame: {0}'.format(index, meanTest.shape))
        
        predictionSuccess.append(gsimNo)
    else: 
        predictionError.append(gsimNo)
        df_error = np.array([99.99, gsimNo])
        dfMissing = pd.DataFrame(data=[df_error], columns=['ERROR', 'gsim.no'])
        dfMissing.to_csv('data/predictions/before/prediction{}_before.csv'.format(index))
        dfMissing.to_csv('data/predictions/before_var/prediction{}_before.csv'.format(index))
        dfMissing.to_csv('data/predictions/test/prediction{}_test.csv'.format(index))
        dfMissing.to_csv('data/predictions/test_var/prediction{}_test.csv'.format(index))
        dfMissing.to_csv('data/predictions/after/prediction{}_after.csv'.format(index))
        dfMissing.to_csv('data/predictions/after_var/prediction{}_after.csv'.format(index))
        print("Size of X_train at df {0} is {1} - to many missing values".format(df['gsim.no'].any(), X_train.shape))

We are at frame: 0


Optimization restart 1/10, f = 28.639935026458986
Optimization restart 2/10, f = 28.639935026459128
Optimization restart 3/10, f = 28.639935026427707
Optimization restart 4/10, f = 28.639935026700357
Optimization restart 5/10, f = 28.639935026435225
Optimization restart 6/10, f = 28.63993503760122
Optimization restart 7/10, f = 28.63993502655177
Optimization restart 8/10, f = 28.639935026489738
Optimization restart 9/10, f = 28.63993502643973
Optimization restart 10/10, f = 28.639935026434642
We are at frame: 1
Size of X_train at df AR_0000002 is (0, 10) - to many missing values
We are at frame: 2


Optimization restart 1/10, f = 3.6159741905198577
Optimization restart 2/10, f = 3.6159741905207525
Optimization restart 3/10, f = 3.6159741907265857
Optimization restart 4/10, f = 3.615974190662483
Optimization restart 5/10, f = 3.615974190522717
Optimization restart 6/10, f = 3.6159741906947303
Optimization restart 7/10, f = 3.6159741905205527
Optimization restart 8/10, f = 3.6159741905223695
Optimization restart 9/10, f = 3.615974190730177
Optimization restart 10/10, f = 3.615974190520821
We are at frame: 3
Size of X_train at df AR_0000004 is (0, 10) - to many missing values
We are at frame: 4


Optimization restart 1/10, f = -173.933336574786
Optimization restart 2/10, f = -173.93333657809595
Optimization restart 3/10, f = -173.93333657509558
Optimization restart 4/10, f = -188.06100560125887
Optimization restart 5/10, f = -188.0610056100172
Optimization restart 6/10, f = -188.06100564605504
Optimization restart 7/10, f = -173.93333656764057
Optimization restart 8/10, f = -188.06100527378683
Optimization restart 9/10, f = -173.93333656311472
Optimization restart 10/10, f = -173.93333656605256
We are at frame: 5


Optimization restart 1/10, f = 69.54250327923658
Optimization restart 2/10, f = 69.54250327854415
Optimization restart 3/10, f = 69.54250327854385
Optimization restart 4/10, f = 69.5425032785968
Optimization restart 5/10, f = 69.54250327864376
Optimization restart 6/10, f = 69.54250327854398
Optimization restart 7/10, f = 69.54250327918649
Optimization restart 8/10, f = 69.54250328610019
Optimization restart 9/10, f = 69.54250327870345
Optimization restart 10/10, f = 69.54250327950962
We are at frame: 6


...

Optimization restart 1/10, f = 35.21318625411372
Optimization restart 2/10, f = 35.21318625400936
Optimization restart 3/10, f = 35.21318626110429
Optimization restart 4/10, f = 35.2131862540076
Optimization restart 5/10, f = 35.21318625401925
Optimization restart 6/10, f = 35.213186254021224
Optimization restart 7/10, f = 35.21318625401624
Optimization restart 8/10, f = 35.2131862540688
Optimization restart 9/10, f = 35.23594521344655
Optimization restart 10/10, f = 35.213186256256975
We are at frame: 419


Optimization restart 1/10, f = 20.40364869979632
Optimization restart 2/10, f = 20.403648684394746
Optimization restart 3/10, f = 20.403648685218435
Optimization restart 4/10, f = 20.403648684240462
Optimization restart 5/10, f = 20.40364871143224
Optimization restart 6/10, f = 20.403648684300673
Optimization restart 7/10, f = 20.4036486842607
Optimization restart 8/10, f = 20.403648684227097
Optimization restart 9/10, f = 20.403648684435947
Optimization restart 10/10, f = 20.403648684246733
We are at frame: 420


Optimization restart 1/10, f = 42.52843501971063
Optimization restart 2/10, f = 42.52843502443027
Optimization restart 3/10, f = 42.5284350197081
Optimization restart 4/10, f = 42.52843501975937
Optimization restart 5/10, f = 42.528435019705086
Optimization restart 6/10, f = 42.52843502007523
Optimization restart 7/10, f = 42.52843534327391
Optimization restart 8/10, f = 42.52843501982521
Optimization restart 9/10, f = 42.528435025475474
Optimization restart 10/10, f = 42.52843501976146
We are at frame: 421


Optimization restart 1/10, f = 42.646500920888556
Optimization restart 2/10, f = 42.64650091741832
Optimization restart 3/10, f = 42.646500917662
Optimization restart 4/10, f = 42.646500918749446
Optimization restart 5/10, f = 42.64650091758206
Optimization restart 6/10, f = 42.646500917440946
Optimization restart 7/10, f = 42.646500917418926
Optimization restart 8/10, f = 42.64650091750845
Optimization restart 9/10, f = 42.64650091741431
Optimization restart 10/10, f = 42.646500918715844
We are at frame: 422


Optimization restart 1/10, f = -55.43083255812708
Optimization restart 2/10, f = -55.430832557625806
Optimization restart 3/10, f = -55.43083255603254
Optimization restart 4/10, f = -55.43083252577455
Optimization restart 5/10, f = -55.43083255803073
Optimization restart 6/10, f = -55.43083255370853
Optimization restart 7/10, f = -55.43083255837068
Optimization restart 8/10, f = -55.43083255718622
Optimization restart 9/10, f = -55.43083255786388
Optimization restart 10/10, f = -55.43083255742431
We are at frame: 423


In [ ]:
#Here we store the training datasets to have them on the harddrive and to be able to merge them later all together without running the whole notebook again. 
for frame, df in enumerate(predictableTimeseries):
    
    gsimNo = df['gsim.no'].values.any()
    splitind = int(df.shape[0] * TRAIN_TEST_THRESHOLD)
    
    datesTrain = df['date'].loc[:splitind -1].copy()
    
    meanTrain = normalizeToGaussian(df['MEAN'].loc[:splitind -1].to_numpy())
    meanTrain = meanTrain.reshape(-1,1)
    dfTest = pd.DataFrame(data=meanTrain.T, columns=datesTrain)
    dfTest['gsim.no'] =gsimNo
    dfTest.to_csv('data/predictions/train/prediction{}_train.csv'.format(frame))
    

In [ ]:
# Loading and merging the previously computed timesegments 
# Then all parts are merged together to creat one single  dataframe with data

for i in range(len(predictableTimeseries)):
    loadSucess = True
    
    try:
        train = pd.read_csv('data/predictions/train/prediction{}_train.csv'.format(i))
    except:
        loadSucess = False
        print('Could not load train data for frame {}'.format(i))
    
    try:    
        test = pd.read_csv('data/predictions/test/prediction{}_test.csv'.format(i))
    except:
        loadSucess = False
        print('Could not load test data for frame {}'.format(i))
    
    try:
        before = pd.read_csv('data/predictions/before/prediction{}_before.csv'.format(i))
    except:
        loadSucess = False
        print('Could not load before data for frame {}'.format(i))
        
    try:
        after = pd.read_csv('data/predictions/after/prediction{}_after.csv'.format(i))
    except:
        loadSucess = False
        print('Could not load after data for frame {}'.format(i))
    
    
    if i == 0:    # initialising the dataset and building it with the first loaded time series
        if (before['gsim.no'].values.any() == train['gsim.no'].values.any() == test['gsim.no'].values.any() == after['gsim.no'].values.any()): 
            before = before.drop(['Unnamed: 0', 'gsim.no'], axis=1)
            train = train.drop(['Unnamed: 0', 'gsim.no'], axis=1)
            test = test.drop(['Unnamed: 0', 'gsim.no'], axis=1)
            after = after.drop(['Unnamed: 0'], axis=1)

            ds = before.join([train, test, after], sort = False)
        
        else: 
            print('Something went wrong in the initialisation step')
            
    else:
        if loadSucess:
            if(before.columns[1] == 'ERROR'):                # check if in the current frame an ERROR occured during prediction 
                ds = ds.append(pd.DataFrame(data= [np.nan]), ignore_index = True) # if so we add a frame full of NaNs, so that we can drop this frame later on
                continue
            
            if (before['gsim.no'].values.any() == train['gsim.no'].values.any() == test['gsim.no'].values.any() == after['gsim.no'].values.any()): # check if they all belong to the same gsim.no and belong to the same frame
                
                before = before.drop(['Unnamed: 0', 'gsim.no'], axis=1)
                train = train.drop(['Unnamed: 0', 'gsim.no'], axis=1)
                test = test.drop(['Unnamed: 0', 'gsim.no'], axis=1)
                after = after.drop(['Unnamed: 0'], axis=1)
                
                try:
                    df = before.join([train, test, after])
                    ds = ds.append(df, ignore_index = True)
                except: 
                    print('Merging of timesegments went wrong in frame {} '.format(i))
                    continue
        else:
            ds = ds.append(pd.DataFrame(data= [np.nan]), ignore_index = True)
            print('Dataframe {} is errorenous and has not been stored correctly'.format(i))


In [ ]:
# Drop autogenerated column and then saving the dataset as a csv
ds = ds.drop([           0], axis=1)
ds.to_csv('data/predictions/ds_gaussian.csv')

In [ ]:
# In this cell we get rid of rows/stations where we could not infer data for. 
# This could be the case if for example all values for the training set were canceled out. 
# Therefore we could not train a GP Regression model and could not infer data. 
# In that case we stored a row with NaN values and the corresponding gsimNumber of that frame. 
# Here we check if all values other than the gsimNumber in the end are null. 
# If so we simply drop them and end up with a those stations we successfully stored data for

for index,row in ds.iterrows():
    if row[:-1].isnull().all():
        ds = ds.drop([index])

In [ ]:
ds.to_csv('data/predictions/ds_gaussian_clean.csv') 